NOTES

1. Tabu list and granularity for fine tuning. 

    2. Granularity should be used for access, not population equality.

3. Change probablity for priotitizing better moves.

    Before:

    After: 

    4. How to pick open facilities? It shouldn't be completely random.

5. Optimize the code. 20k iterations last 70 minutes. And

    Number of Succesful Moves=19404
    
    Last Succesful Move=19999

6.  If we can define a function of travel time in terms of population, can we find a core radius for every district?

7. We should do some collective moves at the beginnning.

8. Reorganize all the code to start using Partition class.


Facility location, redistricting, our problem. Where is our problem, what kind of questions we answer, comparison to what has been done, and how is ours?

Note: Re-organize the code for directed graph. Sort edges in functions.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import plotly.express as px
import plotly.io as pio

from old_bachelor import multi_old_bachelor_seed as old
from data_utils import DataHandler
from data_utils import assign, results

handler = DataHandler()

# Load pre-calculated Objects

In [2]:
"Note: They are loaded in 10 seconds in total."

graph = handler.load_graph() 
candidates = handler.load_candidates() 
chicago = handler.load_chicago()
travel_times = handler.load_travel_times()

# Initial Solution

In [3]:
k_ms = 100
#partitioned_tree, spanning_tree, clusterim, populations, total_travel, open_facilities = old.generate_initial_partition_seed(graph, candidates, travel_times, k_ms)
#initial_data = partitioned_tree, spanning_tree, clusterim, populations, total_travel, open_facilities 
#load.cache_data(initial_data, file_path='prepared_data/initial_data.pkl', method='pickle')
initial_data = handler.load_initial_data()
partitioned_tree, spanning_tree, clusterim, populations, total_travel, open_facilities = initial_data

In [4]:
assign(graph, chicago, clusterim, attribute='initial_cluster', question='is_initial_center')
#plot.districts(chicago, open_facilities, attribute='initial_cluster')

- Consider using partition class properties to reduce computations. (in random_neighbor function and here.)
- #flip(self, flips: Dict) 
- parts: Maps district IDs to the set of nodes in that district.
- Also, see Assignment class. It has "parts" property. {part: <frozenset of nodes in part>}
- from_dict(cls, assignment: Dict) -> "Assignment"
- self.subgraphs = SubgraphView(self.graph, self.parts)

# Search for Initial

In [5]:
"""alpha = 0.01
num_iterations, num_inner_iterations = 1, 20000
granularity = 0.0001
a, b, c = len(graph.nodes)/1000, 1, 1
d_1 = 4
d_2 = 2
omega = 
beta = 
#k_ms = 100  # random 100 facilities are opened among 1000 candidates , total_moves_test
results, initial_solution, final_solution, energy_pop, energy_access, initial_pop, initial_access, last, final_populations, final_total_travel, total_moves = old.multi_old_bachelor_seed(graph, candidates, travel_times, num_iterations, granularity, a, b, c, alpha, num_inner_iterations, k_ms, d_1, d_2, omega, beta, initial_data=initial_data)
4"""

'alpha = 0.01\nnum_iterations, num_inner_iterations = 1, 20000\ngranularity = 0.0001\na, b, c = len(graph.nodes)/1000, 1, 1\nd_1 = 4\nd_2 = 2\nomega = \nbeta = \n#k_ms = 100  # random 100 facilities are opened among 1000 candidates , total_moves_test\nresults, initial_solution, final_solution, energy_pop, energy_access, initial_pop, initial_access, last, final_populations, final_total_travel, total_moves = old.multi_old_bachelor_seed(graph, candidates, travel_times, num_iterations, granularity, a, b, c, alpha, num_inner_iterations, k_ms, d_1, d_2, omega, beta, initial_data=initial_data)\n4'

In [6]:
"""results(graph, initial_solution, final_solution, travel_times, initial_pop, initial_access, energy_pop, energy_access, populations, final_populations, total_travel, final_total_travel, last, total_moves)
"""

'results(graph, initial_solution, final_solution, travel_times, initial_pop, initial_access, energy_pop, energy_access, populations, final_populations, total_travel, final_total_travel, last, total_moves)\n'

In [7]:
"""assign(graph, chicago, final_solution, attribute='cluster', question='is_center')"""
#plot.districts(chicago, candidates, attribute='cluster')

"assign(graph, chicago, final_solution, attribute='cluster', question='is_center')"

# Analysis on Subgraph

In [8]:
" Define two functions for the following two code boxes in the assignment.py class "

"Select Districts"

big_cluster = 24550
small_cluster = 31545

test_nodes = clusterim[big_cluster] + clusterim[small_cluster]
test_subgraph = graph.subgraph(test_nodes)

centers_test = [big_cluster, small_cluster]
test_populations = {small_cluster:populations[small_cluster], big_cluster:populations[big_cluster]}
test_total_travel = {small_cluster: total_travel[small_cluster], big_cluster: total_travel[big_cluster]}
test_initial_solution = {}
test_initial_solution[big_cluster] = clusterim[big_cluster]
test_initial_solution[small_cluster] = clusterim[small_cluster]


In [9]:
"Plot Initial"

subgraph_nodes = list(test_subgraph.nodes())
chicago_test = chicago.loc[subgraph_nodes]

assign(test_subgraph, chicago_test, test_initial_solution, attribute='test_initial_cluster', question='is_test_initial_center')


def plot(data, centers, attribute: str, color=None, fake_center=None):
    # Ensure data[attribute] has appropriate type for indexing
    data[attribute] = pd.Categorical(data[attribute])
    
    # Map each cluster to a color using a cycle of the Plotly qualitative palette
    colors = px.colors.qualitative.Plotly  # This is an example palette
    color_map = {cat: colors[i % len(colors)] for i, cat in enumerate(data[attribute].cat.categories)}
    data['color'] = data[attribute].map(color_map)

    fig = px.choropleth_mapbox(
        data,
        geojson=data.geometry.__geo_interface__,
        locations=data.index,
        color=data['color'],
        mapbox_style="open-street-map",
        center={"lat": data.geometry.centroid.y.mean(), "lon": data.geometry.centroid.x.mean()},
        height=800,
        zoom=10,
        opacity=0.5,
        color_discrete_map="identity",  # Ensure this uses the direct mapping of assigned colors
        hover_data=[data['pop']]  # Show population data on hover
    )

    # Add cluster centers as markers
    for center in centers:
        center_point = data.loc[center].geometry.centroid
        fig.add_scattermapbox(
            lat=[center_point.y],
            lon=[center_point.x],
            mode='markers',
            marker=dict(size=10, color='black'),  # Black markers for centers
            name=f'District={center}'
        )

    return fig.show()

#plot(chicago_test, centers_test, attribute='test_initial_cluster')

In [ ]:
initial_test = partitioned_tree, spanning_tree, test_initial_solution, test_populations, test_total_travel, centers_test


alpha_test = 0
num_iterations_test = 1
num_inner_iterations_test = 30000
granularity_test = 0
a_test = len(test_subgraph.nodes)/100
b_test = 1
c_test = 1
k_test = 2  # random 100 facilities are opened among 1000 candidates 
d_1 = 2
d_2 = 1
omega = 1
beta = 1

results_test, initial_test, final_test, energy_pop_test, energy_access_test, initial_pop_test, initial_access_test, last_test, final_populations_test, final_total_travel_test, total_moves_test, tabu_set = old(test_subgraph, centers_test, travel_times, num_iterations_test, granularity_test, a_test, b_test, c_test, alpha_test, num_inner_iterations_test, k_test, d_1, d_2, omega, beta, initial_data=initial_test)


"Not: Gerrychain hareketli plotu kullan"
#assignment = {node: center for center, nodes in initial_solution.items() for node in nodes}
#from gerrychain import Partition
#partition = Partition(graph, assignment=assignment)

# For the first two iterations:

------ SELECTING NEW MIGRATING NODE ------- Inner Iteration=0

Travel time to Center of Origin=170 Travel Time to Center of Destionation=30

Current Population Energy=0.9978102411776036 Current Access Energy=264725.0094401963

Migrating Node=25933 Neighbor Population Energy=0.9978102411776036 Neighbor Access Energy=264595.0241732923

Worsening Access Energy=267372.25953459827

Threshold=0

Both of the conditions are satisfied.

------ SELECTING NEW MIGRATING NODE ------- Inner Iteration=1

Travel time to Center of Origin=180 Travel Time to Center of Destionation=10

Current Population Energy=0.9978102411776036 Current Access Energy=264595.0241732923

Migrating Node=31537 Neighbor Population Energy=0.9975263835524781 Neighbor Access Energy=264490.0306041486

Worsening Access Energy=267240.9744150252

Threshold=0

Both of the conditions are satisfied.


# For the last iteration:

Migrating Node=24189 Neighbor Population Energy=0.14214170578157156 Neighbor Access Energy=4119.40813810111

Worsening Access Energy=880.5203619292987

Threshold=0.0

None of the conditions are satisfied.

In [11]:
assign(test_subgraph, chicago_test, final_test, attribute='final_initial_cluster', question='is_final_initial_center')

Question 1: Can we remove the selected random endpoints to fasten the algorithm? Define a tabu list. Once we select a valid endpoint, restart the list? Does this provide a significant impact? After a certain number of iterations, it would be useful for fine tunning.

Question 2: Because of the population equality condition, it does not move purple blocks in the left upper side, next to the red highway, to the orange cluster. I think granularity property of old bachelor might solve this issue, since it provides an ordering of good solutions implicitly. Granularity is zero right now. All good solutions are the same.

Final Comment: The search will ignore some good solutions in each iteration to obtain the best solution. A tabu list can help us a lot for not producing the same solution again and again.

NOTES

1. Tabu list and granularity for fine tuning. (Granularity should be used for access, not population equality.)

2. Change probablity for priotitizing better moves.

    Before:

    After: 

3. How to pick open facilities? It shouldn't be completely random.

4. Optimize the code. 20k iterations last 70 minutes. And

    Number of Succesful Moves=19404
    
    Last Succesful Move=19999

5. Should we do some collective moves at the beginnning to fasten the search? 

    - If we can define a function of travel time in terms of population, we can find a 

6. Start with a better initial solution (from recom?).

7. nx.boundary yi surekli hesaplamasak? Input ve return olarak?

8. Initial solution: herkesi en yakin center a assign et. Travel time anlaminda en iyi assignment olacak. Onun uzerinden population equality gelistir. 


In [ ]:
results(test_subgraph, initial_test, final_test, travel_times, initial_pop_test, initial_access_test, 
               energy_pop_test, energy_access_test, test_populations, final_populations_test, test_total_travel, final_total_travel_test, last_test, total_moves_test)
plot(chicago_test, centers_test, attribute='final_initial_cluster')

In [13]:
final_populations_test

{31545: 84913, 24550: 112369}

In [14]:
po = {}

for center in final_test.keys():
    po[center] = sum(graph.nodes[node]['pop'] for node in final_test[center])

po

{24550: 112369, 31545: 84913}

In [15]:
test_populations

{31545: 84913, 24550: 112369}

In [16]:
a = handler.load_initial_data()
par, sp, c, populat, total_tra, open_faci = initial_data

In [17]:
populat[small_cluster]

216

In [18]:
populat[big_cluster]

197066

In [19]:
print(total_tra[small_cluster])
print(total_tra[big_cluster])

50
529600


final: {31545: 173270, 24550: 238430}

# New Initial Solution: Assign nodes to their closest facilities

In [20]:
"""A new initial solution: Assign nodes to their closest facilities"""

closest = {big_cluster:[], small_cluster:[]} 

for c in [small_cluster, big_cluster]:
    for node in clusterim[c]:
        target_cluster = small_cluster if travel_times[(node, small_cluster)] < travel_times[(node, big_cluster)] else big_cluster
        closest[target_cluster].append(node)

In [21]:

test_subgraph = graph.subgraph(test_nodes)
subgraph_nodes = list(test_subgraph.nodes())
chicago_test = chicago.loc[subgraph_nodes]

assign(test_subgraph, chicago_test, test_initial_solution, attribute='test_initial_cluster', question='is_test_initial_center')

In [ ]:
" Define two functions for the following two code boxes in the assignment.py class "

"Select Districts"

closest_nodes = closest[big_cluster] + closest[small_cluster] + [small_cluster, big_cluster]
closest_subgraph = graph.subgraph(closest_nodes)
chicago_closest = chicago.loc[subgraph_nodes]
closest_subgraph_nodes = list(closest_subgraph.nodes)
chicago_closest = chicago.loc[closest_subgraph_nodes]

centers_closest = [big_cluster, small_cluster]
closest_populations = {small_cluster: sum(graph.nodes[node]['pop'] for node in closest[small_cluster]), 
                       big_cluster : sum(graph.nodes[node]['pop'] for node in closest[big_cluster])}

closest_travel = {}
## 

closest_total_travel = {small_cluster: sum(travel_times[(node, small_cluster)] for node in closest[small_cluster]), 
                        big_cluster: sum(travel_times[(node, big_cluster)] for node in closest[big_cluster])}



"Plot Function"

assign(closest_subgraph, chicago_closest, closest, attribute='closest_initial_cluster', question='is_closest_initial_center')
plot(chicago_closest, centers_closest, attribute='closest_initial_cluster')

#results(closest_subgraph, initial_test, final_test, travel_times, initial_pop_test, initial_access_test, 
               #energy_pop_test, energy_access_test, test_populations, final_populations_test, test_total_travel, final_total_travel_test, last_test, total_moves_test)

In [23]:
"Total populations of small and big clusters"

closest_populations 

{31545: 103775, 24550: 93507}

In [24]:
"Total travel times in small and big clusters"

closest_total_travel

{31545: 230490, 24550: 151110}

In [25]:
"Average travel time in small (red) and big (blue) clusters"

closest_total_travel[small_cluster] / len(closest[small_cluster]), closest_total_travel[big_cluster] / len(closest[big_cluster])

(122.53588516746412, 129.81958762886597)

In [26]:
"Maximum travel times in small and big clusters"

furthest_small = max(travel_times[(node, small_cluster)] for node in closest[small_cluster])
furthest_big = max(travel_times[(node, big_cluster)] for node in closest[big_cluster])

furthest_small, furthest_big

(240, 220)

In [27]:
"Maximum travel times in small and big clusters in the previous clustering"

a_furthest_small = max(travel_times[(node, small_cluster)] for node in clusterim[small_cluster])
a_furthest_big = max(travel_times[(node, big_cluster)] for node in clusterim[big_cluster])

a_furthest_small, a_furthest_big

(240, 300)

1. Results are almost perfect. We may have similar results but bad population distribution. 

2. We should minimize maximum travel time and number of blocks that are not assigned to the closest facility, 

3. while we improve population balance (or a function of numbers of D-N teams).

4. Total travel time does not tell us anything meaningful. We should either use max travel time or average travel time.

5. How do we design and integrate selection of candidates?

# A Better and More Natural Algorithm

1. Assign every block to the closest facility for any set of candidates. (cheap and easy)

    Districts are identified and travel time is optimized now. We want to preserve this. 

2. Calculate the number of D-N teams according to the total populations in districts. (cheap and easy)

3. Calculate objective value for the selected candidates. Find a appropriate objective function and maybe some constraints. (cheap and easy)

4. Pick the set of candidates giving the best objective value. If some constraints exist, reject the candidates if constraints fail. (this is the search part)

5. If necessary, we can make a few steps of flips at the end in order to obtain a property we want.

# Thoughts

1. We need to design a meaningful neighborhood for a set of constraints and objective function. 

2. is the connectivity guarenteed? Probably yes. Compactness might be a problem in terms of distance. But we should define compactness in terms of travel time.

    Otherwise, what is the point of travel time? We can still force compactness leaving everything else the same.

3. Orginaze a literature section in the draft, so we can understand if we fill a gap in the literature.

# Components



# Constraints and Objective Value

1. Max travel time from a block to its center should not exceed a constant.

2. D-N teams 

- Generate all_locations

In [28]:
import District
import closest

centers = closest.select_candidates(all_locations)

NameError: name 'all_locations' is not defined